In [2]:
import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib.pyplot as plt
from multiprocessing import Process
import random


Helper Functions

In [1]:
def best_fit_distribution(data, bins=200, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS = [st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,        

        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy
    ]
    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf
    i = 0
    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:
        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    end
                except Exception:
                    pass

                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse

        except Exception:
            pass

    return (best_distribution.name, best_params)
def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

In [3]:
data = pd.read_csv("training_v2.csv")

Drop the rows with more than 90 NaN 

In [4]:
nan_count_col = data.isnull().sum()
list_to_drop_col = []
for col in nan_count_col.index:
    if(nan_count_col[col] > 50000):
        list_to_drop_col.append(col)

In [5]:
data_droped = data.drop(list_to_drop_col,1)
# Patient id and icu id should not have impact on survival rate
data_droped = data_droped.drop('patient_id',1)
data_droped = data_droped.drop('icu_id',1)

In [6]:
nan_count = data_droped.transpose().isnull().sum()
list_to_drop = []
for row in nan_count.index:
    if(nan_count[row] > 57):
        list_to_drop.append(row)


In [7]:
data_droped = data_droped.drop(list_to_drop,0)

In [8]:
# Classify labels into 2 types based on the nature of its contents:
# 1. Numberic Data  2. Labelded Data
# Theses classes could also be helpful for modeling
num_labels = []
lab_labels = []
for col in data_droped.columns:
    if(data_droped[col].nunique() < 30):
        lab_labels.append(col)
    else:
        num_labels.append(col)


In [9]:
num_labels.remove('encounter_id')
num_labels.remove('hospital_id')
lab_labels.append('hospital_id')

Sampling for numberic type data

In [10]:
for label in num_labels:
    # 
    i = 0
    r = 0
    best_fit_name, best_fit_params = best_fit_distribution(data_droped[label].dropna(), 200)
    best_distribution = getattr(st,best_fit_name)
    pdf = make_pdf(best_distribution, best_fit_params)
    while i < data_droped[label].size:
        a = data_droped.index[i]
        if pd.isnull(data_droped[label][a]):
            draw = random.choices(population = pdf.axes[0].to_numpy(),weights = pdf.values,k=1)
            data_droped[label][a] = np.round(draw[0],2)
            r = r + 1
            #print(data_droped[label][a])
        i = i + 1
    #print(label)



Sampling for label type data

In [11]:
for label in lab_labels:
    i = 0
    r = 0
    # print(label)
    while i <data_droped[label].size:
        a = data_droped.index[i]
        if pd.isnull(data_droped[label][a]):
            draw = random.choices(population = data_droped[label].dropna().to_numpy(),k=1)
            data_droped[label][a] = draw[0]
            r = r + 1
        i = i + 1
    # print("{} replaced".format(str(r)))



In [12]:
data_droped.to_csv("filled_training_data.csv")